# Tests

In [1]:
Threads.nthreads()

4

In [2]:
using LinearAlgebra
using Turing
using LimberJack
using PythonCall
using NPZ
using BenchmarkTools;

    CondaPkg Found dependencies: /home/jaime/.julia/environments/v1.7/CondaPkg.toml
    CondaPkg Found dependencies: /home/jaime/.julia/packages/PythonCall/DqZCE/CondaPkg.toml
    CondaPkg Resolving changes
             + libstdcxx-ng
             + python
    CondaPkg Installing packages


  Package            Version  Build               Channel                    Size
───────────────────────────────────────────────────────────────────────────────────
  Install:
───────────────────────────────────────────────────────────────────────────────────

  + jbig                 2.1  h7f98852_2003       conda-forge/linux-64     Cached
  + lz4-c              1.9.3  h9c3ff4c_1          conda-forge/linux-64     Cached
  + olefile             0.46  pyh9f0ad1d_1        conda-forge/noarch       Cached
  + zlib              1.2.12  h166bdaf_4          conda-forge/linux-64     Cached

  Change:
───────────────────────────────────────────────────────────────────────────────────

  - fftw              3.3.10  nompi_hf0379b8_105  conda-forge              Cached
  + fftw              3.3.10  nompi_h77c792f_102  conda-forge/linux-64     Cached
  - openjpeg           2.5.0  h7d73246_1          conda-forge              Cached
  + openjpeg           2.5.0  h7d73246_0          conda-forge/linux-

In [3]:
np = pyimport("numpy");

In [4]:
meta = np.load("../data/SD/SD_meta.npz")
tracers_names = pyconvert(Vector{String}, meta["tracers"])
pairs = pyconvert(Vector{Vector{String}}, meta["pairs"])
pairs_ids = pyconvert(Vector{Vector{Int}}, meta["pairs_ids"])
idx = pyconvert(Vector{Int}, meta["idx"])
data_vector = pyconvert(Vector{Float64}, meta["cls"])
cov_tot = pyconvert(Matrix{Float64}, meta["cov"])
inv_cov_tot = pyconvert(Matrix{Float64}, meta["inv_cov"]);

In [5]:
files = npzread("../data/SD/SD_files.npz");

In [6]:
path = "/home/jaime/PhD/LimberJack.jl/chains/carlos_chains/cl_cross_corr_v3_DES_K1000_all_mag_correctMag/"
pars = np.loadtxt(string(path, "cl_cross_corr_v3_DES_K1000_all_mag_correctMag.bestfit"))
pars = pyconvert(Vector{Float64}, pars);


In [7]:
cls_carlos_ND = np.load(string(path, "cl_cross_corr_bestfit_info.npz"))
cls_carlos_SD = np.load(string(path, "cl_cross_corr_bestfit_info_copy.npz"))
cls_carlos_FD = np.append(cls_carlos_SD["cls"], cls_carlos_ND["cls"])
cls_carlos_FD = pyconvert(Vector{Float64}, cls_carlos_FD)
carlos_chi = pyconvert(Float64, - (cls_carlos_ND["chi2"] +  cls_carlos_SD["chi2"]));

In [8]:
function TTheory(cosmology::Cosmology,
                tracers_names, pairs,
                idx, files;
                Nuisances=Dict())

    ntracers = length(tracers_names)
    tracers = []
    
    nui_type =  eltype(valtype(Nuisances))
    if !(nui_type <: Float64) & (nui_type != Any)
        if nui_type != Real
            cosmology.settings.cosmo_type = nui_type
        end
    end
    
    tracers =  Dict{String}{LimberJack.Tracer}()
    
    for name in tracers_names
        n = length(name)
        t_type = name[n:n]
        println(name)
        if t_type == "0"
            nzs = files[string("nz_", name)]
            nzs = [nzs[i,:] for i in 1:size(nzs,1)]
            zs_mean, nz_mean = nzs[1], nzs[2]

            b = get(Nuisances, string(name, "_", "b"), 1.0)
            nz = get(Nuisances, string(name, "_", "nz"), nz_mean)
            dzi = get(Nuisances, string(name, "_", "dz"), 0.0)
            zs = zs_mean .+ dzi  # Opposite sign in KiDS
            tracer = NumberCountsTracer(cosmology, zs, nz;
                                        b=b)
        elseif t_type == "e"
            nzs = files[string("nz_", name)]
            nzs = [nzs[i,:] for i in 1:size(nzs,1)]
            zs_mean, nz_mean = nzs[1], nzs[2]

            mb = get(Nuisances, string(name, "_", "mb"), 0.0)
            IA_params = [get(Nuisances, "A_IA", 0.0),
                         get(Nuisances, "alpha_IA", 0.0)]
            nz = get(Nuisances, string(name, "_", "nz"), nz_mean)
            dzi = get(Nuisances, string(name, "_", "dz"), 0.0)
            zs = zs_mean .+ dzi  # Opposite sign in KiDS
            tracer = WeakLensingTracer(cosmology, zs, nz;
                                       mb=mb, IA_params=IA_params)
            
        elseif t_type == "k"
            tracer = CMBLensingTracer(cosmology)

        else
            print("Not implemented")
            tracer = nothing
        end
        merge!(tracers, Dict(name => tracer))

    end
    println(tracers)
    npairs = length(pairs)
    total_len = last(idx)
    cls = zeros(cosmology.settings.cosmo_type, total_len)
    for i in 1:npairs
        name1, name2 = pairs[i]
        ls = files[string("ls_", name1, "_", name2)]
        tracer1 = tracers[name1]
        tracer2 = tracers[name2]
        cls[idx[i]+1:idx[i+1]] = angularCℓs(cosmology, tracer1, tracer2, ls)
    end
    
    return cls
end

TTheory (generic function with 1 method)

## Timings

In [10]:
@benchmark make_cls(pars)

BenchmarkTools.Trial: 89 samples with 1 evaluation.
 Range (min … max):  29.678 ms … 109.348 ms  ┊ GC (min … max):  0.00% … 50.86%
 Time  (median):     55.573 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   56.370 ms ±  16.733 ms  ┊ GC (mean ± σ):  11.07% ± 18.53%

  ▂                    ▁ ▁▁█                                    
  █▃▄▁▃▁▁▁▄▁▄▄▃▃▆▁▁▁▄█▇█▃███▇▁▁▃▁▄▃▃▁▃▁▃▃▁▁▄▁▁▁▁▁▁▁▃▁▆▁▄▁▃▁▄▁▃ ▁
  29.7 ms         Histogram: frequency by time         94.9 ms <

 Memory estimate: 29.47 MiB, allocs estimate: 57954.

In [11]:
function diff_cls(pars;
                  tracers_names=tracers_names,
                  pairs=pairs,
                  idx=idx,
                  files=files,
                  data_vector=data_vector,
                  inv_cov=inv_cov_tot)

    Ωm = pars[1] + pars[2]
    s8 = pars[46]
    Ωb = pars[1]
    h = pars[5]
    ns = pars[4]
    
    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, ns, s8, 
                                     tk_mode="emulator",
                                     Pk_mode="Halofit")
    return Theory(cosmology, tracers_names, pairs,
                    idx, files)
end

diff_cls (generic function with 1 method)

In [13]:
function diff_Xi2(pars; data_vector=data_vector, inv_cov=inv_cov_tot)
    nuisances = Dict("DESgc__0_0_b" => pars[6],
                     "DESgc__1_0_b" => pars[7],
                     "DESgc__2_0_b" => pars[8],
                     "DESgc__3_0_b" => pars[9],
                     "DESgc__4_0_b" => pars[10],
                     "DESgc__0_0_dz" => pars[11],
                     "DESgc__1_0_dz" => pars[12],
                     "DESgc__2_0_dz" => pars[13],
                     "DESgc__3_0_dz" => pars[14],
                     "DESgc__4_0_dz" => pars[15],
        
                     "A_IA" => pars[16],
                     "alpha_IA" => pars[17],
        
                     "DESwl__0_e_m" => pars[18],
                     "DESwl__1_e_m" => pars[19],
                     "DESwl__2_e_m" => pars[20],
                     "DESwl__3_e_m" => pars[21],
                     "DESwl__0_e_dz" => pars[22],
                     "DESwl__1_e_dz" => pars[23],
                     "DESwl__2_e_dz" => pars[24],
                     "DESwl__3_e_dz" => pars[25],
        
                     "eBOSS__0_0_b" => pars[26],
                     "eBOSS__1_0_b" => pars[27],

                     "DECALS__0_0_b" => pars[28],
                     "DECALS__1_0_b" => pars[29],
                     "DECALS__2_0_b" => pars[30],
                     "DECALS__3_0_b" => pars[31],
                     "DECALS__0_0_dz" => pars[32],
                     "DECALS__1_0_dz" => pars[33],
                     "DECALS__2_0_dz" => pars[34],
                     "DECALS__3_0_dz" => pars[35],
                    
                     "KiDS1000__0_e_m" => pars[36],
                     "KiDS1000__1_e_m" => pars[37],
                     "KiDS1000__2_e_m" => pars[38],
                     "KiDS1000__3_e_m" => pars[39],
                     "KiDS1000__4_e_m" => pars[40],
                     "KiDS1000__0_e_dz" => pars[41],
                     "KiDS1000__1_e_dz" => pars[42],
                     "KiDS1000__2_e_dz" => pars[43],
                     "KiDS1000__3_e_dz" => pars[44],
                     "KiDS1000__4_e_dz" => pars[45])
    
    Ωm = pars[1] + pars[2]
    s8 = pars[46]
    Ωb = pars[1]
    h = pars[5]
    ns = pars[4]
    
    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, ns, s8, 
                                     tk_mode="emulator",
                                     Pk_mode="Halofit")
    
    theory = Theory(cosmology, tracers_names, pairs,
                    idx, files; Nuisances=nuisances)
    
    diff = @.(theory-data_vector)
    return dot(diff, inv_cov_tot  * diff)

end

diff_Xi2 (generic function with 1 method)

In [14]:
using ForwardDiff
@benchmark ForwardDiff.gradient(diff_Xi2, pars)

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.416 s …   1.453 s  ┊ GC (min … max): 7.61% … 8.25%
 Time  (median):     1.435 s              ┊ GC (median):    8.32%
 Time  (mean ± σ):   1.435 s ± 15.308 ms  ┊ GC (mean ± σ):  8.18% ± 0.40%

  █                     █              █                  █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.42 s         Histogram: frequency by time        1.45 s <

 Memory estimate: 2.28 GiB, allocs estimate: 688383.

In [16]:
using ReverseDiff
ftape_diff_Xi2 = ReverseDiff.GradientTape(diff_Xi2, pars)
compiled_ftape_diff_Xi2 = ReverseDiff.compile(ftape_diff_Xi2)

typename(ReverseDiff.CompiledTape)(diff_Xi2)

In [17]:
Xi2 = similar(pars);

In [18]:
@benchmark ReverseDiff.gradient!(Xi2, compiled_ftape_diff_Xi2, pars)

LoadError: InterruptException:

In [15]:
@benchmark ForwardDiff.jacobian(diff_cls, pars)

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  2.305 s …   2.404 s  ┊ GC (min … max): 8.40% … 8.07%
 Time  (median):     2.340 s              ┊ GC (median):    8.29%
 Time  (mean ± σ):   2.350 s ± 50.542 ms  ┊ GC (mean ± σ):  8.35% ± 0.26%

  █                   █                                   █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.3 s          Histogram: frequency by time         2.4 s <

 Memory estimate: 3.76 GiB, allocs estimate: 1122396.

In [ ]:
@benchmark ReverseDiff.jacobian(diff_cls, pars)

## Xi2

In [43]:
best_pred = make_cls(pars)

610-element Vector{Float64}:
 3.729976949631498e-5
 1.924863264546964e-5
 1.2198839863768642e-5
 2.7338998158421804e-5
 1.8302492387509104e-5
 1.1697676470225297e-5
 8.461577277803604e-6
 6.2847518106000625e-6
 4.953759872071052e-6
 4.0277958603799074e-6
 2.0247593758184622e-5
 1.3879505201281076e-5
 1.0208564507478019e-5
 ⋮
 1.481014723350936e-9
 1.2597256449462859e-9
 1.073704667701235e-9
 9.174815729181471e-10
 7.858916381457959e-10
 6.74149817464629e-10
 5.784141317368861e-10
 4.954936101687788e-10
 4.234671163674685e-10
 3.6091137647921583e-10
 3.0662205257958187e-10
 2.5945231667605537e-10

In [44]:
best_Xi = dot((best_pred .- data_vector), inv_cov_tot  * (best_pred .- data_vector))

726.9778969921368

In [45]:
reduced_best_Xi = round(best_Xi/(length(data_vector)-43), digits=3)

1.282

## Prior sampling

In [21]:
N_samples = 100
cls_preds = zeros(N_samples, length(data_vector))
for i in 1:N_samples
    Ωm = rand(Uniform(0.1, 0.6))
    s8 = rand(Uniform(0.6, 0.9))
    cls_preds[i,:] = make_cls(pars; Ωm=Ωm, s8=s8)
end

In [22]:
cls_preds_m, cls_preds_s = mean(cls_preds, dims=1), std(cls_preds, dims=1);

In [23]:
npzwrite("jaime_cls.npz", Dict("cls_best" => best_pred,
                               "cls_m" => cls_preds_m,
                               "cls_s" => cls_preds_s))

## Turing

In [9]:
@model function model(data_vector::Vector{Float64})
    Ωm ~ Uniform(0.1, 0.6)
    Ωb = 0.05 #~ Uniform(0.03, 0.07)
    h = 0.67 #~ Uniform(0.6, 0.9)
    s8 = 0.81 #~ Uniform(0.6, 1.0)
    ns = 0.96 #~ Uniform(0.87, 1.07)

    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, s8, s8,
                                     tk_mode="emulator",
                                     Pk_mode="Halofit")
    theory = Theory(cosmology,
                    tracers_names, pairs,
                    idx, files)
    data_vector ~ MvNormal(theory, cov_tot)
    return data_vector
end;

In [10]:
iterations = 300
nadapts = 100;

In [11]:
chain3 = sample(model(data_vector), NUTS(5, 0.1; init_ϵ=0.03), iterations,
                progress=true; save_state=true);

┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/iWHPQ/src/hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/iWHPQ/src/hamiltonian.jl:47
Sampling:   1%|▍                                        |  ETA: 0:07:02┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/jaime/.julia/packages/AdvancedHMC/iWHPQ/src/hamiltonian.jl:47
Sampling: 100%|█████████████████████████████████████████| Time: 0:01:28


LoadError: InterruptException:

In [12]:
chain1 = sample(model(data_vector), MH(), iterations,
            nadapt=nadapts, progress=true; save_state=true);

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:24


LoadError: InterruptException:

In [4]:
files = npzread("../data/FD/K1K_DELS_DESY1_eBOSS_CMBk_files.npz")
products = npzread("products.npz")

Dict{String, Array} with 5 entries:
  "idx"         => [0, 5, 13, 23, 34, 47, 52, 57, 62, 67  …  1077, 1099, 1121, …
  "cls_m"       => [0.000229969 7.55142e-5 … 3.71104e-10 3.14893e-10]
  "err"         => [5.919e-5, 1.57008e-5, 6.64192e-6, 3.82552e-6, 2.56294e-6, 2…
  "cls_s"       => [0.000216359 3.16836e-5 … 2.44088e-10 2.0759e-10]
  "data_vector" => [0.000190242, 7.39128e-5, 4.26125e-5, 2.22966e-5, 2.17688e-5…

In [3]:
figure, axis = plt.subplots(5, figsize=(5,14))
l = 1
for i in 0:4
    axis[i+1].errorbar(files[string("ls_1", i, "1", i)], 
                     data[idx[l]:idx[l+1]-1], 
                     yerr=err[idx[l]:idx[l+1]-1], fmt="gs", label="Data")
    axis[i+1].plot(files[string("ls_1", i, "1", i)],
                   cls_preds_m[idx[l]:idx[l+1]-1],
                   "bo-", label="Preds")
    axis[i+1].fill_between(files[string("ls_1", i, "1", i)],
                   cls_preds_m[idx[l]:idx[l+1]-1].+cls_preds_s[idx[l]:idx[l+1]-1],
                   cls_preds_m[idx[l]:idx[l+1]-1].-cls_preds_s[idx[l]:idx[l+1]-1],
                   color="b", alpha=.5)
    axis[i+1].set_title(string("GC", i, "GC", i))
    l += 1
end
plt.legend()
plt.show()
print(l)

LoadError: UndefVarError: files not defined